# TurboPy tutorial

This notebook shows an example app simulating the motion of a block on a spring using the turboPy framework. 

We first need to make the necessary imports:

In [2]:
from turbopy import Simulation, PhysicsModule, Diagnostic, CSVOutputUtility, ComputeTool
import numpy as np

Then create subclasses for the `PhysicsModule` and `Diagnostic` classes and override the necessary methods. These classes hold data necessary for the simulation that will be specified later.

In [3]:
class BlockOnSpring(PhysicsModule):
    """Use turboPy to compute the motion of a block on a spring"""

    def __init__(self, owner: Simulation, input_data: dict):
        super().__init__(owner, input_data)
        self.position = np.zeros((1, 3))
        self.momentum = np.zeros((1, 3))
        self.mass = input_data.get('mass', 1)
        self.spring_constant = input_data.get('spring_constant', 1)
        self.push = owner.find_tool_by_name(input_data["pusher"]).push

    def initialize(self):
        self.position[:] = np.array(self.input_data["x0"])

    def exchange_resources(self):
        self.publish_resource({"Block:position": self.position})
        self.publish_resource({"Block:momentum": self.momentum})

    def update(self):
        self.push(self.position, self.momentum,
                  self.mass, self.spring_constant)

In [4]:
class BlockDiagnostic(Diagnostic):
    def __init__(self, owner: Simulation, input_data: dict):
        super().__init__(owner, input_data)
        self.data = None
        self.component = input_data.get("component", 1)
        self.output_function = None
        self.csv = None

    def inspect_resource(self, resource):
        if "Block:" + self.component in resource:
            self.data = resource["Block:" + self.component]

    def diagnose(self):
        self.output_function(self.data[0, :])

    def initialize(self):
        # setup output method
        functions = {"stdout": self.print_diagnose,
                     "csv": self.csv_diagnose,
                     }
        self.output_function = functions[self.input_data["output_type"]]
        if self.input_data["output_type"] == "csv":
            diagnostic_size = (self.owner.clock.num_steps + 1, 3)
            self.csv = CSVOutputUtility(
                self.input_data["filename"],
                diagnostic_size)

    def finalize(self):
        self.diagnose()
        if self.input_data["output_type"] == "csv":
            self.csv.finalize()

    def print_diagnose(self, data):
        print(data)

    def csv_diagnose(self, data):
        self.csv.append(data)

Then we create a subclass of the `ComputeTool` class to implement the math behind the simulation. For this example, we'll be using the Leapfrog integration method.

In [5]:
class Leapfrog(ComputeTool):
    """Implementation of the leapfrog algorithm

    x_{n+1} = x_n + h * fx(t_{n}, p_{n})
    p_{n+1} = p_n + h * fp(t_{n+1}, x_{n+1})
    """

    def __init__(self, owner: Simulation, input_data: dict):
        super().__init__(owner, input_data)
        self.dt = None

    def initialize(self):
        self.dt = self.owner.clock.dt

    def push(self, position, momentum, mass, spring_constant):
        position[:] = position + self.dt * momentum / mass
        momentum[:] = momentum - self.dt * spring_constant * position

We need to add each of these new classes to the `_registry` attribute for their superclasses.

In [8]:
PhysicsModule.register("BlockOnSpring", BlockOnSpring)
Diagnostic.register("BlockDiagnostic", BlockDiagnostic)
ComputeTool.register("Leapfrog", Leapfrog)

Finally, we create an instance of the main `Simulation` class and run it. The simulation class takes a dictionary of input data as an argument, which contains all the necessary information for the simulation to run. The simulation is designed to save the outputs (time, position, momentum) to csv files.

In [6]:
input_data = {
    "Grid": {"N": 2, "x_min": 0, "x_max": 1},
    "Clock": {"start_time": 0,
              "end_time": 10,
              "num_steps": 100},
    "PhysicsModules": {
        "BlockOnSpring": {
            "mass": 1,
            "spring_constant": 1,
            "pusher": "Leapfrog",
            "x0": [0, 1, 0],
        }
    },
    "Tools": {
        "Leapfrog": {},
    },
    "Diagnostics": {
        # default values come first
        "directory": "output_leapfrog/",
        "output_type": "csv",
        "clock": {"filename": "time.csv"},
        "BlockDiagnostic": [
            {'component': 'momentum', 'filename': 'block_p.csv'},
            {'component': 'position', 'filename': 'block_x.csv'}
        ]
    }
}

In [9]:
sim = Simulation(input_data)
sim.run()

Simulation is initializing
Reading Grid...
Reading Tools...
Reading PhysicsModules...
Reading Diagnostics...
Initializing Simulation Clock...
Initializing Tools...
Initializing PhysicsModules...
Initializing Diagnostics...
Initialization complete
Simulation is started
Simulation complete
